# 综合评价方法  


## 评价问题的五个基本要素
1.评价对象   
2.评价指标  
3.权重系数  
4.综合评价模型   
5.评价者  


### 综合评价体系的构建
评价指标的分类（从指标值的变化对评价目的的影响来看）：
（1）极大型指标（效益型指标）   
（2）极小型指标（成本型指标）  
（3）居中型指标   
（4）区间型指标  

常用的评价指标筛选方法：（用于选取适当的评价指标）    
（1）专家调研法     
（2）最小均方差法   
（3）极大极小离差法（（2）和（3）类似）  

评价指标的预处理方法：   
1.指标的一致化处理（将指标类型统一，如将极小型指标转为极大型指标等等）      
2.指标的无量纲化处理（通过数学变换消除原始指标的单位及其数值数量级的影响，分为归一化和标准化）           
归一化（Normalization）：将数据缩放到[0,1]区间内，如向量归一化。     
标准化（Standardization）：将数据转换为均值为0，标准差为1的分布。       
    (1)标准样本变换法(标准化)
    (2)比例变换法  
    (3)向量归一法  
    (4)极差变换法  
    (5)功效系数法      
     
3.定性指标的定量化（如对于极大型的定性指标定量化时，“很低”可以量化为0，“很高”可以量化为1） 
  

In [3]:
# 例题9.1 评价指标预处理示例

'''
1.首先将x5和x6两项定性指标进行量化处理（不在代码中进行）
2.其中x1,x2,x3为极大型指标
3.费用x4为极小值指标
'''
# 采用了三种无量纲处理的方法得到R1，R2，R3，同时也采用了三种不同的一致化处理方法
# 对于不同的无量纲化处理后的 R1, R2, R3 采用三种不同的一致化处理方法，并不是必须的，示例中是为了展示不同方法的效果
import numpy as np
import pandas as pd

a = np.loadtxt("Pdata9_1_1.txt")
R1 = a.copy(); R2 = a.copy(); R3=a.copy() # 初始化。

# 无量纲化处理 （三种方法都是归一化，不是标准化）
for j in [0,1,2,3,4,5]:
    R1[:,j] = R1[:,j]/np.linalg.norm(R1[:,j]) # 向量归一化法
    R2[:,j] = R2[:,j]/max(R2[:,j]) # 比例变换法
    R3[:,j] = (R3[:,j] - min(R3[:,j])) / (max(R3[:,j]) - min(R3[:,j])) # 极差变换法

# 一致化处理。费用x4为极小值指标,将其转为极大型指标
R1[:,3] = 1-R1[:,3]/np.linalg.norm(R1[:,3])  # 向量归一化后的反转处理 （可以不再进行向量归一化）
R2[:,3] = min(R2[:,3]) / R2[:,3] #倒数变换。原本较小的数（低成本）在取倒数后会变得较大，达到一致化极大型指标的目的。
R3[:,3] = (max(R3[:,3])-R3[:,3]) / (max(R3[:,3])-min(R3[:,3])) #线性变换。首先找到R3第四列的最大值和最小值，然后用最大值减去每个元素，结果除以最大值和最小值的差，实现极小值转换为极大值

# 保存预处理后的数据，给后面使用  np.savetxt 函数会覆盖已有的同名文件，这意味着原来的文件内容会被新的处理结果替换。
np.savetxt("Pdata9_1_2.txt",R1)
np.savetxt("Pdata9_1_3.txt",R2) 
np.savetxt("Pdata9_1_4.txt",R3)

# 生成类型数据
DR1 = pd.DataFrame(R1)
DR2 = pd.DataFrame(R2)
DR3 = pd.DataFrame(R3)
# 写入到excel表格中
with pd.ExcelWriter("Pdata9_1_5.xlsx") as f:  # 使用with语句自动管理资源
    DR1.to_excel(f, "sheet1")  # 把DR1写入到excel的sheet1中
    DR2.to_excel(f, "sheet2")  # 把DR2写入到excel的sheet2中
    DR3.to_excel(f, "sheet3")  # 把DR3写入到excel的sheet3中


## 常用的综合评价数字模型   
综合评价数学模型：将同一评价对象不同方面的多个指标值综合在一起，得到一个整体性评价指标值的数学表达式


### 1.线性加权综合评价模型    
赋权方法：专家咨询主观赋权；熵权法、主成分分析法。   
特点：   
1. 计算简单
2. 权重系数对于评价结果的影响明显     
3. 适用于各评价指标之间相互独立的情况    

### 2.TOPSIS法（理想解的排序方法，technique for order preference by similarity to ideal solution）    
正理想解：一个虚拟的最佳对象，其每个指标值都是所有评价对象中该指标的最好值；      
负理想解：一个虚拟的最差对象，其每个指标值都是所有评价对象中该指标的最差值；   

方法： 求出各评价对象与正理想解和负理想解的距离，并以此对各评价对象进行优劣排序

计算过程：
1. 将评价指标进行预处理（一致化[化为极大型指标]和无量纲化），构造评价矩阵B=(b_ij)_n*m      
2. 确定正理想解C^+和负理想解C^-   
3. 计算各评价对象到正理想解和负理想解的距离 
4. 计算各评价对象对理想解的相对接近度f_i
5. 按照相对接近度f_i 由大到小排列各评价对象的优劣次序



### 灰色关联度分析
具体步骤：
1. 将评价指标进行预处理（一致化处理和无量纲化处理），并构造评价矩阵
2. 确定比较数列b_i（评价对象）和参考数列b_0（评价标准，相当于一个虚拟的最好评价对象的各指标值）
3. 计算灰色关联系数ξ_ij(ξ_ij为比较数列b_i对参考数列b_0在第j个指标上的关联系数)
4. 计算灰色关联度
5. 评价分析（根据灰色关联度的大小，对各评价对象进行排序，关联度越大，评价效果越好）

### 熵值法         
信息量越大，熵值越小，信息的效用值越大；信息量越小，熵值越大，信息的效用值越小。     
熵值法通过计算各指标观测值的信息熵，根据各指标的相对变化程度对系统整体的影响来确定指标权重系数的一种赋权方法。     
计算过程：
1. 计算在第j项指标下第i个评价对象的特征比重
2. 计算第j项指标的熵值（由计算公式可知，第j项指标的观测值差异越大，熵值越小）
3. 计算第j项指标的差异系数（第j项指标的观测值差异越大，则差异系数越大，说明这项指标越重要）
4. 确定第j项指标的权重系数（由差异系数可计算相应的权重系数）
5. 计算第i个评价对象的综合评价值



### 秩和比法（rank sum ratio，RSR） 

基本原理：在一个n行m列矩阵中，通过秩转换，获得无量纲统计量RSR，以RSR值对评价对象的优劣直接排序，从而对评价对象做出综合评价。    
步骤：设置n个评价对象的m个指标的观测值分别为a_ij，得到数据矩阵A=（a_ij）_n*m
1. 编秩（对数据矩阵A逐列编秩，极大性指标从小到大编秩，极小型指标从大到小编秩，指标值相同时编平均秩），得到秩矩阵(R_ij)_n*m
2. 计算秩和比（RSR） 
3. 秩和比排序（根据秩和比RSR对各评价对象进行排序，秩和比越大，评价结果越好）

## 综合评价示例

In [1]:
# 例9.2 
# 利用比例变换法得到的评价矩阵R2作为标准化数据矩阵对战斗机性能进行综合评价（分别用TOPSIS法、灰色关联度、熵值法和秩和比法）

import numpy as np
from scipy.stats import rankdata # 用于计算秩

a = np.loadtxt("Pdata9_1_3.txt")
# 1.TOPSIS法 （确定正理想解+和负理想解；计算各评价对象到正理想解和负理想解的距离；计算各评价对象对理想解的相对接近度；按照相对接近度由大到小排列各评价对象的优劣次序）
cplus = a.max(axis=0) # 逐列求最大值(每个评价指标的最大值)
cminus = a.min(axis=0) # 逐列求最小值(每个评价指标的最小值)
print(f"正理想解：{cplus},负理想解{cminus}")
d1 = np.linalg.norm(a-cplus,axis=1) # 求每个对象（行）与正理想解的距离
d2 = np.linalg.norm(a-cminus,axis=1) # 求每个对象（行）与负理想解的距离
print("每个对象（行）分别与正理想解和负理想解的距离", d1, d2)
f1 = d2 / (d1+d2) #　计算各个评价对象对理想解的相对接近度
print("TOPSIS的评价值为：",f1)

# 2.灰色关联法 (确定参考数列,相当于一个虚拟的最好评价对象的各指标值;计算灰色关联系数;计算灰色关联度;评价分析）
t = cplus-a # 计算参考数列与每个评价对象的差值(差值的绝对大小反映了评价对象与理想状态的偏离程度)
nmin = t.min(); nmax = t.max() # 找出差值的最大值和最小值(用于后续计算关联系数时的标准化处理)
rho = 0.5 # 设定分辨系数为0.5
xs = (nmin+rho*nmax) / (t+rho*nmax) # 计算灰色关联系数
f2 = xs.mean(axis=1) # 等权重,用mean()
print(f"灰色关联系数为: {xs} \n 关联度为：{f2}")

# 3.熵值法（求特征比重矩阵；计算熵值；计算差异系数；计算权重；计算各对象的评价值）
[n,m] = a.shape
cs =a.sum(axis=0) # 求出每列的和
P = a / cs # 求特征比重矩阵
e = -(P*np.log(P)).sum(axis=0) / np.log(n) # 计算熵值
g = 1-e # 计算差异系数
w = g/sum(g) # 计算权重
F = P@w # 计算各对象的评价值
print(f"特征比重矩阵：{P}\n熵值：{e}\n差异系数：{g}\n权重：{w}\n各对象的评价值：{F}")

# 3.秩和比法(编秩；求秩和比RSR；秩和比排序)
R = [rankdata(a[:,i]) for i in np.arange(6)] # 计算每一列的秩
R = np.array(R).T # 构造秩矩阵  # 等价于 R = np.array([a[:,i] for i in np.arange(6)])
RSR = R.mean(axis=1) / n
print(f"秩矩阵为:{R} \n RSR值为：{RSR}")



正理想解：[1. 1. 1. 1. 1. 1.],负理想解[0.72       0.55555556 0.85714286 0.69230769 0.42857143 0.5       ]
每个对象（行）分别与正理想解和负理想解的距离 [0.59538974 0.83163293 0.48540227 0.685136  ] [0.60245103 0.52529122 0.71826658 0.4144692 ]
TOPSIS的评价值为： [0.50294751 0.38711907 0.59673105 0.37692547]
灰色关联系数为: [[0.58823529 0.39130435 0.85714286 0.61111111 0.5        1.        ]
 [1.         1.         0.66666667 0.48148148 0.33333333 0.36363636]
 [0.50505051 0.52427184 1.         1.         1.         0.48780488]
 [0.70422535 0.46153846 0.85714286 0.74074074 0.5        0.36363636]] 
 关联度为：[0.6579656  0.64085297 0.75285454 0.6045473 ]
特征比重矩阵：[[0.23529412 0.1875     0.25316456 0.23990158 0.25       0.37037037]
 [0.29411765 0.3375     0.2278481  0.20299364 0.15       0.18518519]
 [0.21176471 0.25       0.26582278 0.29321304 0.35       0.25925926]
 [0.25882353 0.225      0.25316456 0.26389174 0.25       0.18518519]]
熵值：[0.99468893 0.98294821 0.99888759 0.9936177  0.97032272 0.96836882]
差异系数：[0.00531107 0.01705179 0.00111

## 层次分析法（Analytic Hierarchy Process，简称AHP）

In [9]:
'''
层次分析法:
1.建立结构层次:
    目标层:选择最佳的旅游地.
    准则层:选择的因素或者准则,本题是:景色,费用,饮食,居住和旅途
    方案层:可能的选择方案,本题是:苏杭,黄山和庐山
2.构建判断矩阵:
    在准则层对目标层进行赋权,构建判断矩阵A.
3.计算矩阵A的最大特征值λ和对应的特征向量X,进行归一化处理得到W,W作为权重.
4.方案层对准则层构建判断矩阵:
    对每一个准则层中的因素,都要构建方案层的判断矩阵,得到B1-B5,并以此计算出对应的权重P1-P5.
5.计算组合权向量K,K=PW.
6.评价和排序:根据计算得到的总得分，可以对所有旅游地点进行评价和排序。得分最高的旅游地点被视为最优选择

模型的一致性检验:
    重要性:用来确保通过两两比较得到的判断矩阵在逻辑上是一致的.比如,如果A比B重要2倍,B比C重要2倍,那么A比C应该要重要4倍
    步骤:
        1.计算判断矩阵的特征值,找到最大的特征值λ_max
        2.计算一致性指标(CI)和计算平均随机一致性指标(RI)
        3.进而计算一致性比率(CR): CR=CI / RI
        4.当CR<0.1,则认为通过一致性检验
'''
#  9.3中的例题
from scipy.sparse.linalg import eigs  #eigs: 这是scipy.sparse.linalg模块中的一个函数，用于计算大型稀疏矩阵的前k个特征值和对应的特征向量。函数名eigs来源于英文词"eigenvalues and eigenvectors"的缩写，意思是特征值和特征向量
import numpy as np

a = np.array([[1,1/2,5,5,3],[2,1,7,7,5],[1/5,1/7,1,1/2,1/3],[1/5,1/7,2,1,1/2],[1/3,1/5,3,2,1]]) # 准则层对目标层的判断矩阵A
L,V = eigs(a,1) # 计算判断矩阵A的最大特征值L和对应的特征向量V 
CR = (L-5)/4/1.12 # 计算矩阵A的一致性比率 其中,n=5,CI=1.12
W = V/sum(V) # 归一化处理，得到权重向量W
print('最大的特征值为:',L)
print("最大特征值所对应的特征向量W=\n",W)
print("CR=",CR)

# 定义方案层相对于准则层的判断矩阵B1到B5
B1 = np.array([[1,1/3,1/2],[3,1,1/2],[2,2,1]])
B2 = np.array([[1,3,2],[1/3,1,2],[1/2,1/2,1]])
B3 = np.array([[1,4,3],[1/4,1,2],[1/3,1/2,1]])
B4 = np.array([[1,3,2],[1/3,1,2],[1/2,1/2,1]])
B5 = np.array([[1,2,3],[1/2,1,1/2],[1/3,2,1]])

# 计算每个判断矩阵的最大特征值和对应的特征向量
L1,P1 = eigs(B1,1)
L2,P2 = eigs(B2,1)
L3,P3 = eigs(B3,1)
L4,P4 = eigs(B4,1)
L5,P5 = eigs(B5,1)

# 归一化处理，得到各准则下的方案层权重
P1 = P1/sum(P1)
P2 = P2/sum(P2)
P3 = P3/sum(P3)
P4 = P4/sum(P4)
P5 = P5/sum(P5)

# 修正归一化特征向量的处理方式，使用绝对值函数,消除复数
# P1 = np.abs(P1/sum(P1))  
# P2 = np.abs(P2/sum(P2))
# P3 = np.abs(P3/sum(P3))
# P4 = np.abs(P4/sum(P4))
# P5 = np.abs(P5/sum(P5))

P = np.array([P1,P2,P3,P4,P5])
for i in range(1,6):
    print(f"P{i}={P[i-1]}")

# 计算最终的组合权向量K，作为各方案的最终得分
K = np.hstack([P1,P2,P3,P4,P5]) @ W 
print("K=",K)

最大的特征值为: [5.09758844+0.j]
最大特征值所对应的特征向量W=
 [[0.28626525+0.j]
 [0.48093287+0.j]
 [0.04851865+0.j]
 [0.0685439 +0.j]
 [0.11573932+0.j]]
CR= [0.02178313+0.j]
P1=[[0.16765631]
 [0.34873919]
 [0.4836045 ]]
P2=[[0.54721643]
 [0.26307422]
 [0.18970934]]
P3=[[0.63009766]
 [0.21844266]
 [0.15145968]]
P4=[[0.54721643]
 [0.26307422]
 [0.18970934]]
P5=[[0.54721643]
 [0.18970934]
 [0.26307422]]
K= [[0.44258285+0.j]
 [0.27694046+0.j]
 [0.28047669+0.j]]


In [17]:
# 知乎代码
# 使用了np.round()来四舍五入并格式化输出结果且都只展示了实部

from scipy.sparse.linalg import eigs
from numpy import array, hstack
import numpy as np

a = array([[1, 1 / 2, 5, 5, 3], [2, 1, 7, 7, 5], [1 / 5, 1 / 7, 1, 1 / 2, 1 / 3],
           [1 / 5, 1 / 7, 2, 1, 1 / 2], [1 / 3, 1 / 5, 3, 2, 1]])
L, V = eigs(a, 1)
CR = (L - 5) / 4 / 1.12  # 计算矩阵A的一致性比率
W = V / sum(V)
print("最大特征值为：", np.round(L.real, 4))
print("最大特征值对应的特征向量W=\n", np.round(W.real, 4))
print("CR=", np.round(CR.real, 4))
B1 = array([[1, 1 / 3, 1 / 2], [3, 1, 1 / 2], [2, 2, 1]])
L1, P1 = eigs(B1, 1)
P1 = P1 / sum(P1)
print("P1=", np.round(P1.real, 4))
B2 = array([[1, 3, 2], [1 / 3, 1, 2], [1 / 2, 1 / 2, 1]])
t2, P2 = eigs(B2, 1)
P2 = P2 / sum(P2)
print("P2=", np.round(P2.real, 4))
B3 = array([[1, 4, 3], [1 / 4, 1, 2], [1 / 3, 1 / 2, 1]])
t3, P3 = eigs(B3, 1)
P3 = P3 / sum(P3)
print("P3=", np.round(P3.real, 4))
B4 = array([[1, 3, 2], [1 / 3, 1, 2], [1 / 2, 1 / 2, 1]])
t4, P4 = eigs(B4, 1)
P4 = P4 / sum(P4)
print("P4=", np.round(P4.real, 4))
B5 = array([[1, 2, 3], [1 / 2, 1, 1 / 2], [1 / 3, 2, 1]])
t5, P5 = eigs(B5, 1)
P5 = P5 / sum(P5)
print("P5=", np.round(P5, 4))
K = hstack([P1, P2, P3, P4, P5]) @ W  # 矩阵乘法
print("K=", np.round(K.real, 4))

最大特征值为： [5.0976]
最大特征值对应的特征向量W=
 [[0.2863]
 [0.4809]
 [0.0485]
 [0.0685]
 [0.1157]]
CR= [0.0218]
P1= [[0.1677]
 [0.3487]
 [0.4836]]
P2= [[0.5472]
 [0.2631]
 [0.1897]]
P3= [[0.6301]
 [0.2184]
 [0.1515]]
P4= [[0.5472]
 [0.2631]
 [0.1897]]
P5= [[0.5472+0.j]
 [0.1897+0.j]
 [0.2631+0.j]]
K= [[0.4426]
 [0.2769]
 [0.2805]]
